# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# load data from database
engine = create_engine('sqlite://///content/drive/My Drive/disaster_response/disaster_response/data/DisasterResponse.db')
df = pd.read_sql_table(table_name='DisasterResponse', con=engine)
cat_columns = df.columns[4::]

In [ ]:
df.head()

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):

    # Normalize the text
    text = re.sub(r"\W", " ", text.lower().strip())

    # Tokenize
    words = word_tokenize(text)

    # Remove stop word
    words = [w for w in words if w not in stopwords.words("english")]

    # Stemming and lemmatize
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word).strip() for word in words]

    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 30, max_depth = 5)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X = df['message']
y = df[cat_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
%time pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
(y_pred == y_test).mean().mean()

In [ ]:
for idx, cat in enumerate(cat_columns):
    print('------------------------')
    print('{:}'.format(cat.upper()))
    print(classification_report(y_test[cat], y_pred[:, idx]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'clf__estimator__max_depth': [ 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.